In [1]:
import xml.etree.ElementTree as ET
from xml.sax.saxutils import escape
import pprint
import json
import xmltodict
import pandas as pd
import os

In [2]:
def extractData(inp):
    data=json.loads(jsonString)
    data=data['CHART']['RACE']
    horsieData=[]
    for race in data:
        run={}
        run['distance']=float(race['DISTANCE'])
        run['fr1']=float(race['FRACTION_1'])
        run['fr2']=float(race['FRACTION_2'])
        run['fr3']=float(race['FRACTION_3'])
        run['fr4']=float(race['FRACTION_4'])
        run['fr5']=float(race['FRACTION_5'])
        run['frw']=float(race['WIN_TIME'])
        horse={}
        for entry in race['ENTRY']:
            horse['pp_start']=int(entry['START_POSITION'])
            horse['pp_finish']=int(entry['OFFICIAL_FIN'])
            if(int(entry['OFFICIAL_FIN'])==1):
                horse['first']=1
                horse['second']=0
                horse['third']=0
                horse['loser']=0
            elif(int(entry['OFFICIAL_FIN'])==2):
                horse['first']=0
                horse['second']=1
                horse['third']=0
                horse['loser']=0
            elif(int(entry['OFFICIAL_FIN'])==3):
                horse['first']=0
                horse['second']=0
                horse['third']=1
                horse['loser']=0
            else:
                horse['first']=0
                horse['second']=0
                horse['third']=0
                horse['loser']=1
            horse['age']=int(entry['AGE'])
            horse['speed_rating']=int(entry['SPEED_RATING'])
            horse['horse']=str(entry['AXCISKEY'])
            horse['jockey']=str(entry['JOCKEY']['KEY'])
            horse['trainer']=str(entry['TRAINER']['KEY'])
            horse['odds']=float(entry['DOLLAR_ODDS'])
            call={}
            for poc in entry['POINT_OF_CALL']:
                if(poc['@WHICH']=='1'):
                    call['poc1']=float(poc['LENGTHS'])
    #                 row['poc1p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='2'):
                    call['poc2']=float(poc['LENGTHS'])
    #                 row['poc2p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='3'):
                    call['poc3']=float(poc['LENGTHS'])
    #                 row['poc3p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='4'):
                    call['poc4']=float(poc['LENGTHS'])
    #                 row['poc4p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='5'):
                    call['poc5']=float(poc['LENGTHS'])
    #                 row['poc5p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='FINAL'):
                    call['pocf']=float(poc['LENGTHS'])
    #                 row['pocfp']=int(poc['POSITION'])
            outp={**run, **horse, **call}
            horsieData.append(outp)
    return horsieData

In [3]:
horsieData=[]

for x in os.listdir('resources'):
    file='resources/'+str(x)
    with open(file, 'r') as f:
        xmlString = f.read()
    jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
    horsieData+=extractData(jsonString)

In [4]:
df=pd.DataFrame(horsieData)
df.head()

,age,distance,first,fr1,fr2,fr3,fr4,fr5,frw,horse,...,poc3,poc4,poc5,pocf,pp_finish,pp_start,second,speed_rating,third,trainer
0,4,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049053050051052058055056,...,0.0,0.0,0.0,4.5,4,6,0,107,0,957712
1,6,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049051050051061055061063,...,3.7,0.0,2.0,2.8,3,2,0,110,1,250678
2,5,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049052050052052057061059,...,1.0,0.0,2.1,6.3,6,1,0,104,0,38601
3,7,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049050050052060060061060,...,2.6,0.0,2.7,5.5,5,7,0,105,0,224590
4,4,800.0,1,22.8,46.58,71.25,84.5,0.0,97.5,049053050051054056057063,...,4.7,0.0,2.2,0.0,1,5,0,114,0,6363


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 25 columns):
age             804 non-null int64
distance        804 non-null float64
first           804 non-null int64
fr1             804 non-null float64
fr2             804 non-null float64
fr3             804 non-null float64
fr4             804 non-null float64
fr5             804 non-null float64
frw             804 non-null float64
horse           804 non-null object
jockey          804 non-null object
loser           804 non-null int64
odds            804 non-null float64
poc1            804 non-null float64
poc2            804 non-null float64
poc3            804 non-null float64
poc4            804 non-null float64
poc5            804 non-null float64
pocf            804 non-null float64
pp_finish       804 non-null int64
pp_start        804 non-null int64
second          804 non-null int64
speed_rating    804 non-null int64
third           804 non-null int64
trainer         804 non-n

In [6]:
horseLength=8
furlong=660
dist1=1320
dist2=2640
dist3=3960
df['speed1']=((dist1-(horseLength*df['poc1'].astype(float)))/df['fr1'].astype(float)).round(2)
df['speed2']=((dist2-(horseLength*df['poc2'].astype(float)))/df['fr2'].astype(float)).round(2)
# df['speed3']=((dist3-(horseLength*df['poc3'].astype(float)))/df['fr3'].astype(float)).round(2)
df['speedf']=(((df['distance'].astype(float)/100*furlong)-(horseLength*df['pocf'].astype(float)))/df['frw'].astype(float)).round(2)
df.head()

,age,distance,first,fr1,fr2,fr3,fr4,fr5,frw,horse,...,pocf,pp_finish,pp_start,second,speed_rating,third,trainer,speed1,speed2,speedf
0,4,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049053050051052058055056,...,4.5,4,6,0,107,0,957712,57.89,56.68,53.78
1,6,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049051050051061055061063,...,2.8,3,2,0,110,1,250678,56.81,55.99,53.92
2,5,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049052050052052057061059,...,6.3,6,1,0,104,0,38601,57.72,56.59,53.64
3,7,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049050050052060060061060,...,5.5,5,7,0,105,0,224590,57.02,56.25,53.70
4,4,800.0,1,22.8,46.58,71.25,84.5,0.0,97.5,049053050051054056057063,...,0.0,1,5,0,114,0,6363,56.28,55.82,54.15


In [7]:
# data=pd.DataFrame(df[['distance','pp_start','speed1','speed2','speed3','speedf','pp_finish']])
data=pd.DataFrame(df[['distance','age','jockey','trainer','speed_rating','odds','pp_start','speed1','speed2','speedf','pp_finish']])
data.head()

,distance,age,jockey,trainer,speed_rating,odds,pp_start,speed1,speed2,speedf,pp_finish
0,800.0,4,120035,957712,107,4.4,6,57.89,56.68,53.78,4
1,800.0,6,142413,250678,110,13.9,2,56.81,55.99,53.92,3
2,800.0,5,152588,38601,104,11.1,1,57.72,56.59,53.64,6
3,800.0,7,122644,224590,105,3.9,7,57.02,56.25,53.70,5
4,800.0,4,102032,6363,114,3.1,5,56.28,55.82,54.15,1


In [8]:
data2=pd.DataFrame(df[['distance','age','jockey','trainer','speed_rating','odds','pp_start','speed1','speed2','speedf','pp_finish','first','second','third','loser']])
data2.head()

,distance,age,jockey,trainer,speed_rating,odds,pp_start,speed1,speed2,speedf,pp_finish,first,second,third,loser
0,800.0,4,120035,957712,107,4.4,6,57.89,56.68,53.78,4,0,0,0,1
1,800.0,6,142413,250678,110,13.9,2,56.81,55.99,53.92,3,0,0,1,0
2,800.0,5,152588,38601,104,11.1,1,57.72,56.59,53.64,6,0,0,0,1
3,800.0,7,122644,224590,105,3.9,7,57.02,56.25,53.70,5,0,0,0,1
4,800.0,4,102032,6363,114,3.1,5,56.28,55.82,54.15,1,1,0,0,0


In [9]:
# data.to_csv('output/horseData_20180801v1.csv')
# data2.to_csv('output/horseData_20180801v2.csv', index=False)